In [2]:
import socket
import numpy as np
import matplotlib.pyplot as plt
import math

In [9]:
def le_dados_imu(N, func=None, endereco = "shaka", porta=1234):
    dd = []
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((endereco, porta))
        for i in range(N):
            data = s.recv(32)
            ax = int.from_bytes(data[0:2], byteorder='big', signed=True)
            ay = int.from_bytes(data[2:4], byteorder='big', signed=True) 
            az = int.from_bytes(data[4:6], byteorder='big', signed=True) 
            temp = int.from_bytes(data[6:8], byteorder='big', signed=True) 
            wx = int.from_bytes(data[8:10], byteorder='big', signed=True)
            wy = int.from_bytes(data[10:12], byteorder='big', signed=True)
            wz = int.from_bytes(data[12:14], byteorder='big', signed=True)
            mx = int.from_bytes(data[14:16], byteorder='big', signed=True)
            my = int.from_bytes(data[16:18], byteorder='big', signed=True)
            mz = int.from_bytes(data[18:20], byteorder='big', signed=True)
            t = int.from_bytes(data[-8:], byteorder='little', signed=False)
            dd.append([ax, ay, az, wx, wy, wz, mx, my, mz, temp, t])
            if func:
                func(i)
    return dd

In [ ]:
res2x = np.loadtxt("res2x.txt")
scale = 0.000598
mag_scale = 0.001810
print(res2x)

[-181.4548672  -151.19968288  119.58441564    0.99810697    0.99971396
    0.98932928]


In [13]:
dados = le_dados_imu(6000, lambda i : print("Coletando dados: " + str(int(100*(i+1)/6000))+ "%", end="\r"))


In [25]:
plano1 = np.array ( dados ) [... ,0:3]
np.savetxt("plano1.txt",plano1)

In [27]:
plano2 = np.array ( dados ) [... ,0:3]
np.savetxt("plano2.txt",plano2)

In [29]:
plano3 = np.array ( dados ) [... ,0:3]
np.savetxt("plano3.txt",plano3)

In [31]:
plano4 = np.array ( dados ) [... ,0:3]
np.savetxt("plano4.txt",plano4)

In [12]:
plano5 = np.array ( dados ) [... ,0:3]
np.savetxt("plano5.txt",plano5)

In [15]:
inclinado = np.array ( dados ) [... ,0:3]
np.savetxt("inclinado.txt",inclinado)

In [ ]:
plano1 = np.loadtxt("plano1.txt")
plano2 = np.loadtxt("plano2.txt")
plano3 = np.loadtxt("plano3.txt")
plano4 = np.loadtxt("plano4.txt")
plano5 = np.loadtxt("plano5.txt")
inclinado = np.loadtxt("inclinado.txt")
plano = np.concatenate((plano1,plano2,plano3,plano4))
Q = np.loadtxt("Q.txt")

In [ ]:
def leitura_corr(x,y,z):
    return (x+res2x[0])*res2x[3]*scale,(y+res2x[1])*res2x[4]*scale,(z+res2x[2])*res2x[5]*scale

In [6]:
plano_corr = [leitura_corr(e[0],e[1],e[2]) for e in plano]


In [22]:
media_plano_corr = np.average(plano_corr,0)
norma = np.linalg.norm(media_plano_corr)
k = -media_plano_corr/norma
print(k)

[ 1.18577741e-01 -9.03299841e-04 -9.92944361e-01]


In [55]:
g1 = np.average(np.array([leitura_corr(e[0],e[1],e[2]) for e in plano5]),0)
g2 = np.average(np.array([leitura_corr(e[0],e[1],e[2]) for e in inclinado]),0)
d = g1-g2
j = np.cross(d,k)
j /= np.linalg.norm(j)
i = np.cross(j,k)
Q = np.vstack((i,j,k))
print(Q)
np.savetxt("Q.txt",Q)

[[ 9.92942367e-01  2.30880653e-03  1.18575402e-01]
 [ 2.18540728e-03 -9.99996927e-01  1.17069774e-03]
 [ 1.18577741e-01 -9.03299841e-04 -9.92944361e-01]]


In [43]:
import urllib.request
import json
with urllib.request.urlopen ("http://ip-api.com/json/") as req :
    geodata = json.loads(req.read().decode())
print(geodata)

{'status': 'success', 'country': 'Brazil', 'countryCode': 'BR', 'region': 'SP', 'regionName': 'São Paulo', 'city': 'São Paulo', 'zip': '01000', 'lat': -23.6301, 'lon': -46.6378, 'timezone': 'America/Sao_Paulo', 'isp': 'Universidade De SAO Paulo', 'org': 'Universidade De SAO Paulo', 'as': 'AS28571 UNIVERSIDADE DE SAO PAULO', 'query': '143.107.167.189'}


In [56]:

import ahrs
print(geodata['lat'],geodata['lon'])
wmm = ahrs.utils.WMM()
wmm.magnetic_field(geodata['lat'],geodata['lon'])
F, H, X, Y, Z, I, D = wmm.F, wmm.H, wmm.X, wmm.Y, wmm.Z, wmm.I, wmm.D
print(Z)

-23.6301 -46.6378
-14913.366078721565


In [57]:
dados = le_dados_imu(15000, lambda i : print("Coletando dados: " + str(int(100*(i+1)/15000))+ "%", end="\r"))

In [58]:
np.savetxt("dados_mag_rand.txt",dados)